# K-mer Evolution Notebook

- HDBSCAN github errors!
    - need to find version without problems
    - if now finding one revert back to MA version
- better inclusion of R, N, ... in the kmer
    - status: finished
- evolution on reading frame
    - difficult, ORF notebook necessary for it
- status: unfinished

![Class](Clusterer.svg)

In [1]:
from kneed import DataGenerator, KneeLocator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools as it
from Bio import SeqIO
from Bio.Seq import Seq
import math
import re
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import hdbscan
from scipy.signal import savgol_filter
from sklearn.neighbors import NearestNeighbors
import networkx as nx

In [13]:
k = 7 
split = '|' 
string = ['4', 'Pass'] 
position = [2, 8] 
variable = 0.9
min_clust = 2
sample = 1
num_clust =  60
n_components = 50

In [30]:
class Vectors(object):
    
    def __init__(self, k = 7, split = None, string = [''], position = [0], variable = 0.9):
    
        self.k = k
        self.position = position
        self.split = split
        self.string = string
        self.variable = variable
        self.nucleotides = ['A', 'C', 'G', 'T']
        self.substit = dict.fromkeys(map(ord, self.nucleotides), None)
        self.exist = dict.fromkeys(map(''.join, it.product(self.nucleotides, repeat = self.k)), 0)        
        self.col = len(self.exist.keys())
        
        self.amino = {
            'AAA':'K', 'AAC':'N', 'AAG':'K', 'AAT':'N',
            'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
            'AGA':'R', 'AGC':'S', 'AGG':'R', 'AGT':'S',
            'ATA':'I', 'ATC':'I', 'ATG':'M', 'ATT':'I',
            'CAA':'Q', 'CAC':'H', 'CAG':'Q', 'CAT':'H',
            'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
            'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
            'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
            'GAA':'E', 'GAC':'D', 'GAG':'E', 'GAT':'D',
            'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
            'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
            'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',    
            'TAA':'Y', 'TAC':'*', 'TAG':'*', 'TAT':'Y',
            'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
            'TGA':'*', 'TGC':'C', 'TGG':'W', 'TGT':'C',
            'TTA':'L', 'TTC':'F', 'TTG':'L', 'TTT':'F',
        }
        self.nucex = {
            'A':['A'],
            'C':['C'],
            'G':['G'],
            'T':['T'],
            'R':['A', 'G'],
            'Y':['C', 'T'],
            'W':['A', 'T'],
            'S':['C', 'G'],
            'M':['A', 'C'],
            'K':['G', 'T'],
            'B':['G', 'C', 'T'],
            'H':['A', 'C', 'T'],
            'D':['A', 'G', 'T'],
            'V':['A', 'C', 'G'],
            'N':['A', 'C', 'G', 'T'],
        } 
        self.nucmut = {
            'A':['C', 'G', 'T'],
            'C':['A', 'G', 'T'],
            'G':['A', 'C', 'T'],
            'T':['A', 'C', 'G'],
        } 
    
    def checkup(self, head):
        
        try:
            return(all([re.match(i[0], head[i[1]], re.IGNORECASE) for i in zip(self.string, self.position)]))
        except:
            return(False)
            
    def countRows(self, infile):
        
        row = 0
        for entry in SeqIO.parse(infile,'fasta'):
            
            name = entry.name
            head = name.split(self.split)
            sequence = str(entry.seq)
            missing = len(sequence.translate(self.substit))
            fracture = float(len(sequence)/missing) if missing else 0 
            
            if self.checkup(head) == True and fracture <= self.variable:
                row += 1
                
        return(row)
    
    def calculateFrequence(self, infile):
        
        row = self.countRows(infile)
        index = np.empty(row, dtype = 'object')
        matrix = np.empty((row, self.col, ),dtype = 'float32')
        
        pos = 0
        for entry in SeqIO.parse(infile,'fasta'):
            
            name = entry.name
            head = name.split(self.split)
            sequence = str(entry.seq)
            missing = len(sequence.translate(self.substit))
            fracture = float(len(sequence)/missing) if missing else 0 
            
            if self.checkup(head) == True and fracture <= self.variable:
                for i in range(len(sequence) - self.k + 1):
                    
                    kmer = sequence[i:i+self.k]
                    
                    if fracture == 0:
                        main = [kmer]
                        size = 1
                    else:
                        main = map(''.join, it.product(*[self.nucex.get(j) for j in kmer]))
                        size = np.prod([len(self.nucex.get(k)) for k in kmer])
                        
                    for sub in main:
                        self.exist[sub] += float(1/size)
                    #    self.exist[sub] += float((1-self.mutfac)/size)     
                    #    for l, nuc in enumerate(sub):
                    #        for mutation in map(''.join, it.product(*[[sub[:l]], self.nucmut.get(nuc), [sub[l+1:]]])):
                    #            self.exist[mutation] += float(self.mutfac/(size*12))

                matrix[pos] = np.fromiter(self.exist.values(), dtype = 'float32', count = self.col)/sum(self.exist.values())
                index[pos] = head[0]
                
                self.exist.update((k,0) for k in self.exist.keys())
                pos += 1
            
        return(index, matrix)

In [41]:
def Cluster(linkage, min_clust, num_clust):

    x = 0.0
    y = 1.0
    cluster = linkage.get_clusters(cut_distance = x, min_cluster_size = min_clust)
    n = cluster.max().item()

    while n != num_clust:

        if n < num_clust and n != -1:
            x = x - y
            y = y * 0.1

        else:
            x = x + y

        cluster = linkage.get_clusters(cut_distance = x, min_cluster_size = min_clust)
        n = cluster.max().item()
        
    return(cluster)

In [33]:
vectors = Vectors(k = k, split = split, string = string, position = position, variable = variable)
index, matrixl1 = vectors.calculateFrequence('A.fasta')

In [37]:
pca = PCA(n_components = n_components)
matrixpca = pca.fit_transform(matrixl1)
variance = pca.explained_variance_ratio_.sum()

In [38]:
matrixl2 = normalize(matrixpca, norm='l2')

In [39]:
hdbinit = hdbscan.HDBSCAN(min_samples = sample, min_cluster_size = min_clust, gen_min_span_tree = True, metric = 'euclidean').fit(matrixl2)

In [40]:
linkage = hdbinit.single_linkage_tree_

In [42]:
cluster = Cluster(linkage, min_clust, num_clust)

In [50]:
pd.DataFrame(zip(index, cluster), columns = ['accession', 'cluster']).set_index('accession').to_csv('cluster.csv', index=True, header=True, sep=',', mode='w')
linkage.to_pandas().set_index('parent', inplace = False).to_csv('linkage.csv', index=True, header=True, sep=',', mode='w')